In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy import sparse
from scipy.spatial import distance
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Rates are constructed as
$$ \omega_{\mathrm{ij}}=\frac{2 \pi}{\hbar}\left|J_{\mathrm{ij}}\right|^2 \sqrt{\frac{1}{4 \lambda k_{\mathrm{B}} T}} \exp \left(-\frac{\left(\lambda+\Delta E_{\mathrm{ij}}\right)^2}{4 \lambda k_{\mathrm{B}} T}\right) $$

The resistances are therefore constructed as
$$ R_{\mathrm{ij}}= \frac{\Delta E_{\mathrm{ij}}}{Q_{e} \cdot  \omega_{\mathrm{ij}}} $$

In [2]:
def Construct(Coo, Typ, Ener, J=0.003, T=300, Lam=0.2):
    #Read coordinates from file
    coords = list(map(tuple, np.genfromtxt(Coo, delimiter=" ")))
    #Build Graph
    F=nx.Graph()
    F.add_nodes_from(coords)
    #Read Attributes
    types = np.genfromtxt(Typ, delimiter=" ")
    energies = np.genfromtxt(Ener, delimiter=" ")
    #Set Attributes
    i=0
    for u in F.nodes():
        F.nodes[u]["pos"]=u
        F.nodes[u]["type"]=types[i]
        F.nodes[u]["potential"]=energies[i]
        i+=1  
    #Construct Edges
    F.add_edges_from(nx.geometric_edges(F, 1))
    
    ## Inizializing Marcus Rates ##
   
    for (u, v) in F.edges(): #die beiden Endpunkte u und v der Kante 
        #from settings file
        #J=J ## attempt frequency (largest possible rate) =>  "maximum ti"
        #Lam =  Reorganization Energy = lambda = materials => l
        kbT=0.0000861801*T #eV
        H_Bar = 6.58264*(10**(-16)) # Planck in eV
        Qe = 1.602176634*(10**(-19)) # Electron charge
        
        #dist=distance.euclidean(F.nodes[u]["pos"] , F.nodes[v]["pos"]) ## distantce between hopping sites
        
        deltaE=F.nodes[u]["potential"] - F.nodes[v]["potential"] ## v is start and u is end
        rate=2 * np.pi / H_Bar * np.abs(J)**2 * np.sqrt(1/(4 * Lam * kbT)) * np.exp(-((Lam + deltaE)**2)/(4* Lam * kbT))
        resistance = deltaE / (Qe * rate )
        
        if deltaE <0: ## flow from high potential to low
            F.edges[u,v]['weight'] = resistance
        else: ## reorient edge
            # Could not just delete edge and add oposite as (Non-Di-)Graphs sort Edges by the sequence of nodes
            # Therefore I just add a negative sign to the resistor and thereby flipping the current direction
            F.edges[u,v]['weight'] = (-1) * resistance
    
    return F


#Gitter=Construct("coord_0.dat", "mol_types_0.dat", "site_energies_0.dat")

In [3]:
class Solve:
    def __init__(self, X):
        self.G=X
        self.N=self.G.order()
        self.nR=self.G.number_of_edges()
        
    def resistor(self):
        #Extract the values of the resistors from the graph and build a nR x nR matrix
        mat=sparse.spdiags(list(nx.get_edge_attributes(self.G, "weight").values()), 0, self.nR, self.nR)
        return sparse.csc_matrix(mat)
    
    def incidence(self):
        #Builds the incidence matrix from the graph
        mat= np.transpose(nx.incidence_matrix(self.G, oriented=1)) #Beachte Transpose damit die Dimensionen der networkx funktion zum paper passen
        
        
        ## Siehe Codeschnipsel im ersten Beispiel falls die Orientierung ungünstig ist ##
        
        
        return mat
    
    def voltages(self):
        #Get the potential values from the nodes and build a vector
        vec=np.array(list(nx.get_node_attributes(self.G, "potential").values()))
        return vec

    
    def currents(self):
        #Combines the other functions to get the currents trough the resistors
        return - (sparse.linalg.inv(self.resistor()) @ self.incidence()) @ self.voltages()

    
#Solve(Gitter).currents()   